In [ ]:
import sys

#!{sys.executable} -m pip install gensim==3.8.3
#!{sys.executable} -m pip install fse

In [ ]:
import os, re, gzip, json, pickle, shutil, random, joblib

import numpy as np

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

from fse.models import SIF
from fse import IndexedList

from sklearn.manifold import TSNE
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
data_path = '../data'